图论的最小生成树问题，就是让你从图中找若干边形成一个边的集合 mst，这些边有以下特性：

1、这些边组成的是一棵树（树和图的区别在于不能包含环）。
2、这些边形成的树要包含所有节点。
3、这些边的权重之和要尽可能小。


首先，Kruskal 算法用到了贪心思想，来满足权重之和尽可能小的问题：

！先对所有边按照权重从小到大排序，从权重最小的边开始，选择合适的边加入 mst 集合，这样挑出来的边组成的树就是权重和最小的。
其次，Kruskal 算法用到了 Union-Find 并查集算法，来保证挑选出来的这些边组成的一定是一棵「树」，而不会包含环或者形成一片「森林」：

如果一条边的两个节点已经是连通的，则这条边会使树中出现环；如果最后的连通分量总数大于 1，则说明形成的是「森林」而不是一棵「树」。


首先，Prim 算法也使用贪心思想来让生成树的权重尽可能小，也就是「切分定理」。

其次，Prim 算法使用 BFS 算法思想 和 visited 布尔数组避免成环，来保证选出来的边最终形成的一定是一棵树。

Prim 算法不需要事先对所有边排序，而是利用优先级队列动态实现排序的效果，所以我觉得 Prim 算法类似于 Kruskal 的动态过程。

下面以一道题为例
1584. 连接所有点的最小费用
https://leetcode-cn.com/problems/min-cost-to-connect-all-points/

In [2]:
# Kruskal算法
# https://labuladong.gitee.io/algo/2/20/42/
def f(points: list):
    sz = len(points)

    # union-find 模板
    count = sz
    roots = list(range(count))
    rank = [1] * count

    def find(x):
        if roots[x] == x:
            return x
        else:
            roots[x] = find(roots[x])
            return roots[x]

    def union(x, y):
        nonlocal count
        rx, ry = find(x), find(y)
        if rx != ry:
            if rank[rx] < rank[ry]:
                roots[ry] = rx
            elif rank[ry] < rank[rx]:
                roots[rx] = ry
            else:
                roots[rx] = ry
                rank[ry] += 1
            count -= 1

    # 1.建立节点的cost排序
    costs = []
    for i in range(sz - 1):
        for j in range(i + 1, sz):
            cost = abs(points[i][0] - points[j][0]) + abs(points[i][1] - points[j][1])
            costs.append((cost, i, j))
    costs.sort()
    # 2.创建最小生成树
    mst = 0
    for cost, i, j in costs:
        if find(i) == find(j):  # 2.1成环跳过
            continue
        union(i, j)  # 2.2不成环连接作为最小生成树
        mst += cost
    # 3.检查联通分量 看是否存在孤岛
    return mst if count == 1 else -1


f([[0, 0], [2, 2], [3, 10], [5, 2], [7, 0]])


20

In [5]:
# Prim算法
# https://labuladong.gitee.io/algo/2/20/43/
import heapq


def f(points: list):
    sz = len(points)
    # 1.建立邻接表
    neis = [[] for _ in range(sz)]
    for cur in range(sz):
        for nei in range(sz):
            if cur != nei:
                cost = abs(points[cur][0] - points[nei][0]) + abs(points[cur][1] - points[nei][1])
                neis[cur].append((nei, cost))
    # 2.准备工作 优先队列 最小生成树集合 切分函数
    pq = []
    inMst = {0}
    mst = 0

    def cut(node): # 切分函数 把该点邻接边切掉==邻接点入队 已经在最小生成树集合的跳过
        for nei, cost in neis[node]:
            if nei not in inMst:
                heapq.heappush(pq, (cost, nei))
    # 3.任意找一个节点先切
    cut(0)
    # 4.开始循环切分
    while pq:
        # 4.1先找成本最低的点作为最小生成树一部分 如果已经在最小生成树就跳过 防止成环
        curCost, cur = heapq.heappop(pq)
        if cur in inMst:
            continue
        inMst.add(cur)
        mst += curCost
        # 4.2对新加入的最小生成树的点进行切分
        cut(cur)
    # 5.检查是否有孤岛
    return mst if len(inMst) == sz else -1


f([[0, 0], [2, 2], [3, 10], [5, 2], [7, 0]])


20